تحميل مكتبة تحويلات الجمل

In [ ]:
pip install sentence_transformers

كود قراءة البيانات من مجموعة البيانات stsbenchmark

In [3]:
"""
This example runs a CNN after the word embedding lookup. The output of the CNN is than pooled,
for example with mean-pooling.


"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import csv
import gzip


#### Just some code to print debug information to stdout
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)
#### /print debug information to stdout

# Read the dataset
batch_size = 32
model_save_path = "output/training_stsbenchmark_cnn-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# Check if dataset exists. If not, download and extract  it
sts_dataset_path = "datasets/stsbenchmark_ar.tsv"


logging.info("Read STSbenchmark train dataset")

train_samples = []
dev_samples = []
test_samples = []
with open(sts_dataset_path, "rt", encoding="utf8") as fIn:
    reader = csv.DictReader(fIn, delimiter="\t", quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row["score"]) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row["sentence1"], row["sentence2"]], label=score)

        if row["split"] == "dev":
            dev_samples.append(inp_example)
        elif row["split"] == "test":
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)


كود قراءة مجموعة البيانات (AR-ASAG) ودمجها مع مجموعة البيانات (sts banchmark ) 

In [ ]:
# يتم استخدام هذه الطريق في حال أردنا دمج مجموعتي البيانات من أجل تدريب النموذج

import pandas as pd
import numpy as np

dataset_path = "datasets/dataset_ar.xlsx"
df = pd.read_excel(dataset_path)
df = df.dropna()

main = []
for i in df.iterrows():
  inp_example = InputExample(texts=[i[1]["Teacher_Answer"], i[1]["Answer"]], label=i[1]['Degree'] / 100)
  main.append(inp_example)


train_samples.extend(main[:700])
test_samples.extend(main[700:980])
dev_samples.extend(main[980:1180])

print(len(main[700:980]))
print(len(train_samples) , len(test_samples) , len(dev_samples))

280
6449 1659 1700


كود تحميل النموذج وبدء التدريب

In [ ]:
# Map tokens to vectors using BERT

# النماذج المستخدمة
word_embedding_model = models.Transformer("bert-base-uncased")
# word_embedding_model = models.Transformer("google-bert/bert-base-multilingual-cased")
# word_embedding_model = models.Transformer("distilbert/distilbert-base-uncased")
# word_embedding_model = models.Transformer("distilbert/distilbert-base-multilingual-cased")
# word_embedding_model = models.Transformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

cnn = models.CNN(
    in_word_embedding_dimension=word_embedding_model.get_word_embedding_dimension(),
    out_channels=256,
    kernel_sizes=[1, 3, 5],
)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(
    cnn.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)


model = SentenceTransformer(modules=[word_embedding_model, cnn, pooling_model])


# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name="sts-dev")

# Configure the training
num_epochs = 10
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name="sts-test")
model.evaluate(evaluator)


 كود استدعاء النموذج وقياس نسبة التشابة بين نصين

In [10]:
# بعد عملية التدريب سيتم حفظ النموذج في مجلد بالاسم الذي تم تحديده أعلى وفي حال اردنا اعادة استخدام هذا النوذج نقوم باستخدام هذا الجزء وتعريف المدل مع تعريب المكتبة التي تم تعريفها اعلى

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer(model_save_path)



text1 = "الإنترنت هي نظام عالمي واسع النطاق من شبكات الكمبيوتر المترابطة. تتكون من شبكات أصغر، مثل شبكات الشركات والجامعات والحكومات، متصلة ببعضها البعض باستخدام مجموعة متنوعة من تقنيات الاتصال، مثل خطوط الهاتف، والكابلات، والاتصالات اللاسلكية."
text2 = "الإنترنت هي شبكة عالمية تربط بين أجهزة الكمبيوتر في جميع أنحاء العالم. تسمح هذه الشبكة بتبادل المعلومات والبيانات، مثل النصوص والصور والصوت والفيديو، بين المستخدمين. يمكن الوصول إلى الإنترنت من خلال مجموعة متنوعة من الأجهزة، بما في ذلك أجهزة الكمبيوتر المحمولة، والهواتف الذكية، والأجهزة اللوحية."



def embed(text):
  embedding = model.encode(text, convert_to_tensor=True)
  return embedding

def the_result(t1,t2):
  embedding1 = embed(t1)
  embedding2 = embed(t2)

  cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
  return round(float(cosine_similarity),2)

the_result(text1,text2)


0.68

كود قراءة مجموعة البيانات AR-ASAG

In [ ]:
import pandas as pd
import numpy as np
b_data = pd.read_excel('datasets/dataset_ar.xlsx')
b_data = b_data.dropna()
num_raw = 1180
sentences1 = b_data['Teacher_Answer'][num_raw:].tolist()
sentences2 = b_data['Answer'][num_raw:].tolist()
degree = b_data['Degree'][num_raw:]
y_pre = []
y = [i / 100 for i in degree]
y2 = y[::-1]
xx = -1
for i, j in zip(sentences1, sentences2):
    yy = round(the_result(i, j) , 2)
    y_pre.append(yy)
    print(f'index :{xx * -1},model Score: {yy}, Teacher Score: {round(y2[xx],2)}')
    xx -= 1

كود حساب نسبة الارتباط ونسبة الخطأ ودقة النموذج بعد قراءة البيانات من مجموعة البيانات 

In [ ]:

# حساب نسبة الإرتباط
cor = np.corrcoef(y, y_pre)
print(cor)

# حساب نسبة الخطأ
n = 0
for y1, y2 in zip(y, y_pre):
    n = n + 1 if (y1 - y2) >= 0.3 or (y2 - y1) >= 0.3 else n
percent = n / (len(b_data) / 100)
print(f"%{percent}")


# حساب الدقة
from sklearn.metrics import accuracy_score

yt_round = list(map(lambda x: round(x), y))
yp_round = list(map(lambda x: round(x), y_pre))

n = 0
y10 = []
for y1, y2 in zip(yt_round, yp_round):
    n = n + 1 if y1==y2 else n
print(f"%{accuracy_score(yt_round,yp_round) * 100}")
